*This script connects to Google Ad Manager API, downloads key-values for campaign, transforms it to lower case and uploads it back.*


---



# Steps:
1. To get access to Google Ad Manager API you have to Generate OAuth Credentials following the instruction in [the official documentation](https://developers.google.com/ad-manager/api/authentication#service).
Use instruction steps for "**Service account**" authentication type. Generate a service account ID and JSON key file.
2. Run all cells one by one (`Ctrl+F9` or `Cmd+F9`) and follow instructions for each cell
3. To make an actual update on CELL 4, review all previous steps and approve the operation by entering `yes`

In [ ]:
# @title **CELL 1**: Upload Generated Json key file by clicking button ***Choose files*** bellow
#@markdown Run cell and upload generated Json key file by clicking button **`Choose files`** bellow
!pip install --upgrade googleads -q

from google.colab import files
import json
import os

# Verify first uploaded file
def verify_file(uploaded):
  file_name = next(iter(uploaded.keys()))
  with open(file_name) as f:
    try:
      data = json.loads(uploaded[file_name])
    except json.JSONDecodeError:
      print("Error: The uploaded file is not valid JSON.")
      exit(1)

    if data.get("type") != "service_account":
      print("Error: The uploaded file is not a service account key file.")
      exit(1)
    else:
      print('Key file "{name}" is valid'.format(name=file_name))

uploaded = files.upload()

verify_file(uploaded)

# Get the first uploaded file
uploaded_file = next(iter(uploaded.keys()))

# Get the file path by joining the current working directory and the uploaded file name
file_path = os.path.join(os.getcwd(), uploaded_file)

# Print the file path
# print(file_path)

In [ ]:
#@title **CELL 2**: Connect to Google Ad Manager
from googleads import ad_manager
from googleads import oauth2

API_VERSION = 'v202402'

# Ad Manager API information.
APPLICATION_NAME = 'fd-ad-manager-test'

def get_client(network_code=None):
  oauth2_client = oauth2.GoogleServiceAccountClient(
        file_path, oauth2.GetAPIScope('ad_manager'))

  ad_manager_client = ad_manager.AdManagerClient(
      oauth2_client, APPLICATION_NAME, network_code=network_code)
  return ad_manager_client

def choose_network(ad_manager_client):
  network_service = ad_manager_client.GetService('NetworkService', version=API_VERSION)

  networks = network_service.getAllNetworks()

  if len(networks) > 1:
    # Print all networks with index
    index = 1
    for network in networks:
        print(f"{index}. Network with network code '{network['networkCode']}' and display name '{network['displayName']}' was found.")
        index += 1

    # Ask user to choose the index of the desired network
    chosen_index = int(input("\nChoose the index of the desired network: "))

    # Validate the chosen index
    if chosen_index < 1 or chosen_index > len(networks):
        print("\nInvalid index. Please rerun cell and choose a valid index.")
    else:
        # Get the chosen network
        chosen_network = networks[chosen_index - 1]
  else:
    chosen_network = networks[0]

  print("Chosen network has network code '%s' and display name '%s'." %
          (chosen_network['networkCode'], chosen_network['displayName']))
  return chosen_network

def print_info(ad_manager_client):
  current_network = ad_manager_client.GetService('NetworkService').getCurrentNetwork()
  current_user = ad_manager_client.GetService('UserService').getCurrentUser()
  print(f"\nCurrent network:\n{current_network}")
  print(f"\nCurrent user:\n{current_user}")

# Get client for current network
ad_manager_client = get_client()
chosen_network = choose_network(ad_manager_client)
ad_manager_client = get_client(network_code=chosen_network['networkCode'])

print_info(ad_manager_client)

In [ ]:
#@title **CELL 3**: Find non-lowercase values for `searchparams` key
KEY_NAME_FOR_VALUES_UPDATE = 'searchparams' # @param {type: "string"}
# #@markdown *Cange `KEY_NAME_FOR_VALUES_UPDATE` if it differs from the key containing search values*
REVIEW_UPPERCASE_VALUES = True # @param {type: "boolean"}
# #@markdown *Check `REVIEW_UPPERCASE_VALUES` if you want to see and review all duplicated values*
FIELD_NAME = 'name'

def find_key_id_by_name(client, key_name):
  # Initialize appropriate service.
  custom_targeting_service = client.GetService(
      'CustomTargetingService', version=API_VERSION)

  statement = (ad_manager.StatementBuilder(version=API_VERSION)
                .Where('name = :name OR displayName = :displayName')
                .WithBindVariable('name', key_name)
                .WithBindVariable('displayName', key_name)
                )
  # Get custom targeting keys by statement.
  response = custom_targeting_service.getCustomTargetingKeysByStatement(
      statement.ToStatement())
  # print(response)
  if 'results' in response and len(response['results']):
    key_id = response['results'][0]['id']
  else:
    print(f"Key {key_name} wasn't found")
    exit(1)
  return key_id

def get_values_to_update(client, key_id):
  # Initialize appropriate service.
  custom_targeting_service = client.GetService(
      'CustomTargetingService', version=API_VERSION)

  statement = (ad_manager.StatementBuilder(version=API_VERSION)
               .Where('customTargetingKeyId = :keyId')
               .WithBindVariable('keyId', int(key_id)))

  values = []
  while True:
    response = custom_targeting_service.getCustomTargetingValuesByStatement(
        statement.ToStatement())

    if 'results' in response and len(response['results']):
      values.extend(response['results'])
      statement.offset += statement.limit
    else:
      break

  if response['totalResultSetSize'] == 0:
    print('No custom targeting values were found.')
  return values

def is_value_lowercase(value):
  return value.lower() == value.replace("%2B", "%2b")


def get_non_lowercase_values(values, field):
  non_lowercase_count = 0
  values_to_fix = []

  for value in values:
      # for field in fields_to_check:
        if field in value and value[field] and not is_value_lowercase(value[field]):
            non_lowercase_count += 1
            values_to_fix.append(value)

  return values_to_fix

def lower_value(value):
  return value.lower().replace("%2b", "%2B")

def print_values(values):
    counter = 1
    for value in values:
      value_name = value[FIELD_NAME]
      value_id = value['id']
      print(f"[ID: {value_id}] {value_name} -> {lower_value(value_name)}")

def review_upper(values):
  # dupe_review = input(f"\nWould you like to review each value duplicating by field \"{field}\"? (yes/no): ").lower()
  # if dupe_review in ["y", "ye", "yes"] :
  if REVIEW_UPPERCASE_VALUES and values:
    print('\nReview changes to values:')
    print_values(values)


key_id = find_key_id_by_name(ad_manager_client, KEY_NAME_FOR_VALUES_UPDATE)
values = get_values_to_update(ad_manager_client, key_id)
print(f"Values found for key {KEY_NAME_FOR_VALUES_UPDATE}: {len(values)}")

values_to_fix = get_non_lowercase_values(values, FIELD_NAME)
print(f"Number of all non lowercase values by name: {len(values_to_fix)}")
review_upper(values_to_fix)

# print(values_to_fix)

In [ ]:
#@title **[DEACTIVATED] CELL 4**: ~~Update `Name` of values to lowercase~~
#@markdown **Warning!** id of values will be changed thus such values will be removed from active campaigns

def recreate_value(value, key_id):
  new_value = [
      {
          'customTargetingKeyId': value['customTargetingKeyId'],
          'name': lower_value(value['name']),
          'displayName': value['displayName'],
          'matchType': value['matchType']
      }
  ]
  print(value)
  print(new_value)

  custom_targeting_service = ad_manager_client.GetService(
      'CustomTargetingService', version=API_VERSION)

  action = {'xsi_type': 'DeleteCustomTargetingValues'}
  value_statement = (ad_manager.StatementBuilder(version=API_VERSION)
                         .Where('customTargetingKeyId = :keyId '
                                'AND id = :valueId')
                         .WithBindVariable('keyId', key_id)
                         .WithBindVariable('valueId', value['id']))

  result = custom_targeting_service.performCustomTargetingValueAction(
      action, value_statement.ToStatement())
  # print(result)

  if result and int(result['numChanges']) == 1:
    print(f"Value {value[FIELD_NAME]} was deleted")
    # print(f"{get_values_to_update(ad_manager_client, key_id)}")
    values = custom_targeting_service.createCustomTargetingValues(new_value)
    print(f"Value was added: {values}")
    # print(f"{get_values_to_update(ad_manager_client, key_id)}")
  else:
    print(f"Value {value[FIELD_NAME]} was NOT deleted")
    exit(1)

# for value in values_to_fix:
#   recreate_value(value, key_id)
# recreate_value(values_to_fix[0], key_id)


In [ ]:
#@title **[DEACTIVATED] CELL 5**: ~~Update `displayName` of values to lowercase~~
def update_values_displayName_to_lowercase(client, key_id):
  # Initialize appropriate service.
  custom_targeting_service = client.GetService(
      'CustomTargetingService', version=API_VERSION)

  statement = (ad_manager.StatementBuilder(version=API_VERSION)
               .Where('customTargetingKeyId = :keyId')
               .WithBindVariable('keyId', int(key_id)))

  while True:
    # Get custom targeting values by statement.
    response = custom_targeting_service.getCustomTargetingValuesByStatement(
        statement.ToStatement())

    # Update each local custom targeting value object by changing its display name to lowercase.
    if 'results' in response and len(response['results']):
      updated_values = []
      for value in response['results']:
        if value['displayName'] and value['displayName'] != value['displayName'].lower():
          value['displayName'] = value['displayName'].lower()
          updated_values.append(value)
      values = custom_targeting_service.updateCustomTargetingValues(
          updated_values)

      # Display results.
      print (f"Number of updated values: {len(values)}")
      for value in values:
        print('Custom targeting value with id "%s", name "%s", and display'
              ' name "%s" was updated.'
              % (value['id'], value['name'], value['displayName']))
      statement.offset += statement.limit
    else:
      break

  if response['totalResultSetSize'] == 0:
    print('No custom targeting values were updated.')


# update_approval = input("Are you sure you want to update all displayName of values to lowercase? (yes/no): ").lower()

# if update_approval in ["y", "ye", "yes"] :
#     update_values_displayName_to_lowercase(ad_manager_client, key_id)
# else:
#     print("Operation cancelled.")